# Итераторы

## Порядок сдачи домашнего

Под каждое домашнее вы создаете отдельную ветку куда вносите все изменения в рамках домашнего. Как только домашнее готово - создаете пулл реквест (обратите внимание что в пулл реквесте должны быть отражены все изменения в рамках домашнего). Ревьювера назначаете из таблицы - https://docs.google.com/spreadsheets/d/1vK6IgEqaqXniUJAQOOspiL_tx3EYTSXW1cUrMHAZFr8/edit?gid=0#gid=0
Перед сдачей проверьте код, напишите тесты. Не забудьте про PEP8, например, с помощью flake8. Задание нужно делать в jupyter notebook.

**Дедлайн - 11 ноября 10:00**

## Итератор по цифрам

Реализуйте класс-итератор `DigitIterator`, который принимает на вход целое число и позволяет итерироваться по его цифрам слева направо. На каждой итерации должна возвращаться следующая цифра числа.

**Условия:**
1.	Число может быть как положительным, так и отрицательным.
2.	Итератор должен возвращать только цифры числа, без знака - для отрицательных чисел.
3.	Итерация должна быть возможна с помощью цикла for или функции next().

**Пример использования:**

```python
iterator = DigitIterator(12345)
for digit in iterator:
    print(digit)
# 1
# 2
# 3
# 4
# 5

iterator = DigitIterator(-6789)
for digit in iterator:
    print(digit)

# 6
# 7
# 8
# 9
```

In [21]:
class DigitIterator:
    def __init__(self, number):
        self.number = []
        number = abs(number)
        length = len(str(number))
        while length:
            self.number.append(number % 10)
            number //= 10
            length -= 1

    def __iter__(self):
        return self

    def __next__(self):
        if self.number:
            return self.number.pop()
        raise StopIteration


# Пример использования
iterator = DigitIterator(-12345)
for digit in iterator:
    print(digit)

1
2
3
4
5


# Итератор по файлу чанками

Реализуйте класс-итератор `FileChunkIterator`, который принимает на вход путь к файлу и количество байт для чтения. Итератор должен открывать файл и считывать его содержимое блоками фиксированного размера (количества байт), переданного в качестве параметра. При каждой итерации возвращается следующий блок байт, пока не будет достигнут конец файла.

**Условия:**
1.	Итератор должен открывать файл в режиме чтения бинарных данных (rb).
2.	Размер блока (количество байт) передаётся при создании итератора.
3.	Если в конце файла остаётся блок меньшего размера, итератор должен вернуть оставшиеся байты.
4.	При достижении конца файла итератор должен завершить работу, поднимая StopIteration.

**Пример использования:**
```python
with open("example.txt", "w") as file:
    file.write("Hello world!!")
    
iterator = FileChunkIterator("example.txt", 2)
for chunk in iterator:
    print(chunk)
# He
# ll
# o
# wo
# rl
# d!
# !
```

In [22]:
class FileChunkIterator:
    def __init__(self, file_path, chunk_size):
        self.chuck_size = chunk_size
        self.file = open(file_path, 'rb')

    def __iter__(self):
        return self

    def __next__(self):
        result = self.file.read(self.chuck_size)
        if result:
            return result.decode()
        raise StopIteration

Комментарий к заданию:
нет возможности закрыть файл, так как если в методе `__next__` при окончании чтения файла вызвать `self.file.close()`, то при следующем вызове `next` мы поймаем ошибку, отличную от `StopIteration` (это не удовлетворяет интерфейсу итератора).

Я вижу в этом проблему, так как ресурс не будет освобожден.

In [26]:
# Реализация с закрытием файла
class FileChunkIterator:
    def __init__(self, file_path, chunk_size):
        self.chuck_size = chunk_size
        self.position = 0
        self.file_path = file_path

    def __iter__(self):
        return self

    def __next__(self):
        with open(self.file_path, 'rb') as file:
            file.seek(self.position)
            result = file.read(self.chuck_size)
            self.position += self.chuck_size
            if result:
                return result.decode()
            raise StopIteration

In [27]:
with open("example.txt", "w") as file:
    file.write("Hello world!!")

In [28]:
iterator = FileChunkIterator("example.txt", 2)
for chunk in iterator:
    print(chunk)

He
ll
o 
wo
rl
d!
!


# Итератор по подматрицам

Реализуйте класс-итератор `SubmatrixIterator`, который принимает на вход матрицу и размер подматрицы (квадратного блока). Итератор должен проходить по всем возможным подматрицам указанного размера и возвращать их одну за другой.

**Пример использования:**

```python
matrix = [
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 15, 16]
]
iterator = SubmatrixIterator(matrix, 2)
for submatrix in iterator:
    print(submatrix)
    
# [[1, 2], [5, 6]]
# [[2, 3], [6, 7]]
# [[3, 4], [7, 8]]
# [[5, 6], [9, 10]]
# [[6, 7], [10, 11]]
# [[7, 8], [11, 12]]
# [[9, 10], [13, 14]]
# [[10, 11], [14, 15]]
# [[11, 12], [15, 16]]
```

In [29]:
class SubmatrixIterator:
    def __init__(self, matrix, submatrix_size):
        self.matrix = matrix
        self.matrix_size = (len(matrix), len(matrix[0] if len(matrix) else 0))
        self.submatrix_size = submatrix_size
        self.pos = (0, 0)

    def __iter__(self):
        return self

    def _is_valid_offset(self):
        return all(self.pos[i] + self.submatrix_size <= self.matrix_size[i]
                   for i in range(2))

    def _next_position(self):
        x, y = self.pos
        if y + 1 + self.submatrix_size <= self.matrix_size[1]:
            return (x, y + 1)
        return (x + 1, 0)

    def __next__(self):
        if self._is_valid_offset():
            i_range = range(self.pos[0], self.pos[0] + self.submatrix_size)
            j_range = range(self.pos[1], self.pos[1] + self.submatrix_size)
            submatrix = [[self.matrix[i][j] for j in j_range] for i in i_range]
            self.pos = self._next_position()
            return submatrix
        raise StopIteration

In [30]:
matrix = [
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 15, 16]
]
iterator = SubmatrixIterator(matrix, 2)
for submatrix in iterator:
    print(submatrix)

[[1, 2], [5, 6]]
[[2, 3], [6, 7]]
[[3, 4], [7, 8]]
[[5, 6], [9, 10]]
[[6, 7], [10, 11]]
[[7, 8], [11, 12]]
[[9, 10], [13, 14]]
[[10, 11], [14, 15]]
[[11, 12], [15, 16]]


# Построчного чтение всех файлов в директории

Реализуйте класс-итератор  `RecursiveFileLineIteratorNoHidden`, который принимает на вход путь к директории и рекурсивно проходит по всем файлам, включая файлы во вложенных директориях. Итератор должен возвращать строки из каждого файла построчно, игнорируя файлы и директории, названия которых начинаются с точки (.), т.е. скрытые файлы и папки.

**Условия:**
1.	Итератор должен проходить по всем файлам в указанной директории и всех её поддиректориях, кроме тех, что начинаются с точки (.).
2.	Итератор должен возвращать строки из каждого файла поочерёдно, построчно.
3.	Поддерживаются только текстовые файлы.
4.	После завершения чтения всех файлов итератор должен завершить работу, поднимая StopIteration.
5.	Обработайте ситуацию, если файл не может быть открыт (например, из-за ошибок доступа).

**Пример использования:**

```python
iterator = RecursiveFileLineIteratorNoHidden("./test")
for line in iterator:
    print(line)
    
# Example 1
# Example 2
# Example 3
# Example 4
# Subfolder Example 1
# Subfolder Example 2
# Subfolder Example 3
# Subfolder Example 4    
```

Для выполнения задания потребуются несколько методов из модуля os, которые позволяют работать с файловой системой в Python. Давайте подробно рассмотрим их.


`os.walk(top, topdown=True, onerror=None, followlinks=False)` — это генератор, который рекурсивно обходит директории и поддиректории, начиная с указанного пути top. На каждом шаге возвращается кортеж, содержащий текущую директорию, список поддиректорий и список файлов.

Возвращаемые значения:
* root: Текущая директория, в которой находимся в данный момент обхода.
* dirs: Список поддиректорий в текущей root директории.
* files: Список файлов в текущей root директории.

`os.path.join(path, *paths)` объединяет один или несколько компонентов пути, возвращая корректный путь, соответствующий операционной системе. Это полезно для построения путей к файлам и директориям в кросс-платформенном формате.

```python
root = "/path/to/directory"
file_name = "example.txt"
full_path = os.path.join(root, file_name)
print(full_path)  # Вывод: "/path/to/directory/example.txt"
```

`os.path.isfile(path)` проверяет, является ли указанный путь файлом. Возвращает True, если path указывает на файл, и False, если это директория или объект другого типа.

```python
file_path = "/path/to/file.txt"
if os.path.isfile(file_path):
    print("Это файл.")
else:
    print("Это не файл.")
```

`os.path.basename(path)` возвращает базовое имя файла или директории из пути. Это полезно, если нужно получить только имя файла или папки, без остальных компонентов пути.

```python
file_path = "/path/to/file.txt"
print(os.path.basename(file_path))  # Вывод: "file.txt"
```

`os.path.isdir(path)` проверяет, является ли указанный путь директорией. Возвращает True, если path указывает на директорию, и False, если это файл или объект другого типа.

```python
dir_path = "/path/to/directory"
if os.path.isdir(dir_path):
    print("Это директория.")
else:
    print("Это не директория.")
```

In [31]:
import os


class RecursiveFileLineIteratorNoHidden:
    def __init__(self, root):
        self.current_file = None
        self.queue = []
        self._collect_files(root)
        self.queue.reverse()

    def __iter__(self):
        return self

    def _collect_files(self, root):
        for current, subfolders, files in os.walk(root):
            subfolders[:] = [folder for folder in subfolders
                             if not folder.startswith('.')]
            for filename in files:
                if not filename.startswith('.'):
                    self.queue.append(os.path.join(current, filename))

    def _close_current_file(self):
        self.current_file.close()
        self.current_file = None

    def __next__(self):
        while True:
            if not self.current_file:
                if self.queue:
                    try:
                        self.current_file = open(self.queue.pop(), 'r')
                    except OSError:
                        continue
                else:
                    raise StopIteration

            try:
                result = self.current_file.readline()
                if not result:
                    self._close_current_file()
                else:
                    return result.strip('\n')
            except IOError:
                self._close_current_file()

In [32]:
iterator = RecursiveFileLineIteratorNoHidden("./test")
for line in iterator:
    print(line)